In [ ]:
#!pip install torch transformers datasets pandas openpyxl scikit-learn
# Uninstall existing transformers and torch to avoid conflicts
!pip uninstall -y transformers torch
# Install specific versions for compatibility
!pip install numpy==1.26.4 torch==2.3.0 transformers==4.44.2 datasets pandas openpyxl scikit-learn
# Verify installations
import sys
import torch
import transformers
import pandas
import datasets
import sklearn

print("Python version:", sys.version)
print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)
print("Pandas version:", pandas.__version__)
print("Datasets version:", datasets.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("GPU available:", torch.cuda.is_available())

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch version: 2.6.0+cu124
Transformers version: 4.52.4
Pandas version: 2.2.2
Datasets version: 2.14.4
Scikit-learn version: 1.6.1
GPU available: True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

# Get the notebook's directory
notebook_path = '/content/drive/My Drive/Colab Notebooks'
file_path = os.path.join(notebook_path, 'T-HSAB.xlsx')

# Load the dataset
try:
    data = pd.read_excel(file_path, header=None, names=['text', 'label'])
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Check the file path and ensure T-HSAB.xlsx is in {notebook_path}.")
    raise

# Inspect the dataset
print("Dataset Preview:")
print(data.head())
print("\nColumn Names:")
print(data.columns)
print("\nUnique Labels:")
print(data['label'].unique())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset Preview:
                                                text   label
0  اسغي ياشعب تونس تدعوا بالاسلام كفار الحمدلله ن...    hate
1  قطع يد السارق توفرت الشروط شرط الحد الأدنى قيم...  normal
2                             تلوموش لطفي لعبدلي شرف  normal
3  مستغرب شعب يسمع تفاهة شانو لى الدرجة الشعب تاف...  normal
4  هههخ غزلتني مافهمتش شمدخلها الموضوع تتنطر وحده...  normal

Column Names:
Index(['text', 'label'], dtype='object')

Unique Labels:
['hate' 'normal' 'abusive']


Step 3: Preprocess the Data

Map labels to integers and split the data into training (80%) and validation (20%) sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Map labels to integers
label_mapping = {'normal': 0, 'abusive': 1, 'hate': 2}
data['label'] = data['label'].map(label_mapping)

# Check for unmapped labels
print("Missing labels after mapping:", data['label'].isna().sum())
if data['label'].isna().sum() > 0:
    print("Unmapped labels found:", data[data['label'].isna()]['label'].unique())
    raise ValueError("Update label_mapping to include all labels.")

# Split the data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")

Missing labels after mapping: 0
Training samples: 4819, Validation samples: 1205


Step 4: Convert to Hugging Face Dataset

Convert to Dataset format.

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

Step 5: Load Model and Tokenizer

Use TunBERT for Tunisian dialect.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "tunis-ai/TunBERT"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping))
except Exception as e:
    print(f"Error loading model/tokenizer: {e}")
    raise

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tunis-ai/TunBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0

Step 6: Tokenize the Text

Tokenize with a maximum length of 128.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

Map:   0%|          | 0/1205 [00:00<?, ? examples/s]

Step 7: Define Training Arguments

Save outputs to Google Drive.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/Colab Notebooks/results",
    eval_strategy="epoch",  # Updated from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir='/content/drive/My Drive/Colab Notebooks/logs',
    logging_steps=10,
    run_name="tunbert-t-hsab-finetune",
    report_to="none",
)

Step 8: Define Evaluation Metrics

Compute accuracy, precision, recall, and F1-score

In [ ]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

Step 9: Train the Model

Fine-tune the model.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.512500,1.768994,0.502905,0.587560,0.502905,0.509480
2,0.523100,1.243610,0.546058,0.594518,0.546058,0.545691
3,0.381600,1.288543,0.629046,0.594840,0.629046,0.588669


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.512500,1.768994,0.502905,0.587560,0.502905,0.509480
2,0.523100,1.243610,0.546058,0.594518,0.546058,0.545691
3,0.381600,1.288543,0.629046,0.594840,0.629046,0.588669
4,0.394200,1.382690,0.590041,0.579033,0.590041,0.583796


In [ ]:
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 0.8565424680709839, 'eval_accuracy': 0.6381742738589211, 'eval_precision': 0.6139852628289075, 'eval_recall': 0.6381742738589211, 'eval_f1': 0.6178453113902986, 'eval_runtime': 8.5219, 'eval_samples_per_second': 141.4, 'eval_steps_per_second': 17.719, 'epoch': 6.0}


In [ ]:
model.save_pretrained("/content/drive/My Drive/Colab Notebooks/fine_tuned_model")
tokenizer.save_pretrained("/content/drive/My Drive/Colab Notebooks/fine_tuned_model")

('/content/drive/My Drive/Colab Notebooks/fine_tuned_model/tokenizer_config.json',
 '/content/drive/My Drive/Colab Notebooks/fine_tuned_model/special_tokens_map.json',
 '/content/drive/My Drive/Colab Notebooks/fine_tuned_model/vocab.txt',
 '/content/drive/My Drive/Colab Notebooks/fine_tuned_model/added_tokens.json',
 '/content/drive/My Drive/Colab Notebooks/fine_tuned_model/tokenizer.json')

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="/content/drive/My Drive/Colab Notebooks/fine_tuned_model", tokenizer="/content/drive/My Drive/Colab Notebooks/fine_tuned_model")
test_text = "اسغي ياشعب تونس تدعوا بالاسلام كفار"
result = classifier(test_text)
label_reverse_mapping = {0: 'normal', 1: 'abusive', 2: 'hate'}
predicted_label = label_reverse_mapping[int(result[0]['label'].split('_')[-1])]
print(f"Prediction: {predicted_label}, Confidence: {result[0]['score']:.4f}")

In [ ]:
from fastapi import FastAPI
from transformers import pipeline

app = FastAPI()
classifier = pipeline("text-classification", model="/content/drive/My Drive/Colab Notebooks/fine_tuned_model")

@app.post("/predict")
async def predict(text: str):
    result = classifier(text)
    label = label_reverse_mapping[int(result[0]['label'].split('_')[-1])]
    return {"text": text, "label": label, "confidence": result[0]['score']}